In [1]:
import pandas as pd
from pathlib import Path
from datetime import datetime
from xml.etree.ElementTree import iterparse

In [2]:
data_dir = Path('../../../data/stackoverflow/')

In [3]:
questions = []
for _, node in iterparse(data_dir / "raw/gamedev_questions.xml", events=("end",)):
    if node.tag == "row":
        questions.append({
            'id': node.attrib.get('Id'),
            'accepted_id': node.attrib.get('AcceptedAnswerId'),
            'post_date': node.attrib.get('CreationDate'),
            'author_id': node.attrib.get('OwnerUserId'),
            'title': node.attrib.get('Title'), 
            'text': node.attrib.get('Body'),
        })
    node.clear()
questions = pd.DataFrame(questions)

answers = []
for _, node in iterparse(data_dir / "raw/gamedev_answers.xml", events=("end",)):
    if node.tag == "row":
        answers.append({
            'id': node.attrib.get('Id'),
            'question_id': node.attrib.get('ParentId'),
            'post_date': node.attrib.get('CreationDate'),
            'author_id': node.attrib.get('OwnerUserId'),
            'text': node.attrib.get('Body')
        })
    node.clear()
        
answers = pd.DataFrame(answers)

comments = []
for _, node in iterparse(data_dir / "raw/gamedev_comments.xml", events=("end",)):
    if node.tag == "row":
        comments.append({
            'id': node.attrib.get('Id'),
            'parent_id': node.attrib.get('PostId'),
            'post_date': node.attrib.get('CreationDate'),
            'author_id': node.attrib.get('UserId'),
            'text': node.attrib.get('Text')
        })
        node.clear()
        
comments = pd.DataFrame(comments)

In [4]:
dateparse = lambda t: datetime.strptime(t, '%Y-%m-%dT%H:%M:%S.%f')

In [5]:
comments = comments.drop_duplicates('id')
comments.post_date = comments.post_date.apply(dateparse)
comments = comments.merge(answers[['id', 'question_id']].rename(columns={'id': 'parent_id'}), on='parent_id', how='left')
comments.loc[comments.question_id.isna(), 'question_id'] = comments[comments.question_id.isna()]['parent_id']
comments = comments[['id', 'author_id', 'question_id', 'post_date', 'text']]

In [6]:
answers = answers.drop_duplicates('id')
answers.post_date = answers.post_date.apply(dateparse)
answers['is_accepted'] = answers.id.isin(questions.accepted_id).apply(lambda n: 1 if n else 0)
answers = answers[['id', 'author_id', 'question_id', 'post_date', 'is_accepted', 'text']]

In [7]:
questions = questions.drop_duplicates('id')
questions.post_date = questions.post_date.apply(dateparse)
questions['is_resolved'] = questions.accepted_id.apply(lambda n: 0 if n is None else 1)
questions['url'] = questions.id.apply(lambda s: f'https://stackoverflow.com/questions/{s}/index.html')
questions = questions[['id', 'author_id', 'post_date', 'is_resolved', 'title', 'text', 'url']]

In [8]:
users = pd.concat([df.author_id.reset_index() for df in [questions, comments, answers]])
users = users[['author_id']].drop_duplicates()
users = users.rename(columns={'author_id': 'id'})
users['url'] = users.id.apply(lambda s: f'https://stackoverflow.com/users/{s}/index.html')

In [9]:
comments.to_parquet(data_dir / 'unprocessed/comments.parquet', index=False)
answers.to_parquet(data_dir / 'unprocessed/answers.parquet', index=False)
questions.to_parquet(data_dir / 'unprocessed/questions.parquet', index=False)
users.to_parquet(data_dir / 'users.parquet', index=False)